In [33]:
import re

In [34]:
test_string='''94.19.17.65 - - [13/Dec/2015:13:41:09 +0100] "GET /administrator/ HTTP/1.1" 200 4263 "-" "Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0" "-"'''

In [35]:
http_log_matcher = r'^([0-9.]+)[^[]+\[([^:]+):([0-9]{2}:[0-9]{2}:[0-9]{2})[^]]+]\s+"([^"]+)"\s+(\d+)\s+(\d+)\s+"-"\s+"([^"]+)'

In [36]:
re.match(http_log_matcher, test_string).groups()

('94.19.17.65',
 '13/Dec/2015',
 '13:41:09',
 'GET /administrator/ HTTP/1.1',
 '200',
 '4263',
 'Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0')

In [37]:
# not OK request?
data = open("../data/http-accesslog.txt")
line = data.readline()
badreq=0
while line:
    matched = re.match(http_log_matcher, line)
    if(matched):
        ip,date,time,request,status,length,browser = matched.groups()
        if(int(status) >= 400):
            badreq+=1
    line = data.readline()
print("There were", badreq, " failures in the file")
data.close()

There were 145  failures in the file


In [38]:
# redirects
data = open("../data/http-accesslog.txt")
line = data.readline()
redirect=0
while line:
    matched = re.match(http_log_matcher, line)
    if(matched):
        ip,date,time,request,status,length,browser = matched.groups()
        if(int(status) // 100 == 3):
            redirect+=1
            print(line)
    line = data.readline()
print("There were", redirect, " redirects in the file")
data.close()

188.163.80.234 - - [15/Dec/2015:23:24:31 +0100] "GET /administrator HTTP/1.1" 301 256 "-" "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:41.0) Gecko/20100101 Firefox/41.0" "-"

188.163.80.234 - - [17/Dec/2015:18:48:54 +0100] "GET /administrator HTTP/1.1" 301 256 "-" "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:41.0) Gecko/20100101 Firefox/41.0" "-"

There were 2  redirects in the file


In [39]:
# Busiest day
data = open("../data/http-accesslog.txt")
days={}
lineno=0
line = data.readline()

while line:
#     if lineno<5:
#         print("line:",line)
#         lineno+=1
    matched = re.match(http_log_matcher, line)
    if(matched):
        ip,date,time,request,status,length,browser = matched.groups()
        if(date in days):
            days[date] = days[date]+1
        else:
            days[date]=1
    line = data.readline()
print("There are", len(days), " days in the file")
biggest = 0
bigday=""
for day in days:
    if(days[day]>biggest):
        biggest = days[day]
        bigday=day
print("The busiest day was", bigday)
data.close()

There are 22  days in the file
The busiest day was 19/Dec/2015


In [40]:
# broswer analysis
# Busiest day
data = open("../data/http-accesslog.txt")
browsers={}
platforms={}
lineno=0
line = data.readline()

while line:
#     if lineno<5:
#         print("line:",line)
#         lineno+=1
    matched = re.match(http_log_matcher, line)
    if(matched):
        ip,date,time,request,status,length,agent = matched.groups()
        browser_match = re.search("Firefox/([0-9.]+)", agent)
        if browser_match:
            version = browser_match.group(1)
            if version in browsers:
                browsers[version]+=1
            else:
                browsers[version]=1
            os_match = re.search("\((\w+)", agent)
            if os_match:
                os = os_match.group(1)
                if os in platforms:
                    platforms[os]+=1
                else:
                    platforms[os]=1
    line = data.readline()
print("There are", len(browsers), " in the file")
biggest = 0
popular=""
for version in browsers:
    if(browsers[version]>biggest):
        popular=version
print("The most popular was", popular)
print("They ran on", len(platforms), "platforms")
print(platforms)
data.close()

There are 29  in the file
The most popular was 1.5.0.1
They ran on 4 platforms
{'Windows': 2858, 'X11': 33, 'Macintosh': 4, 'Android': 4}
